# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import json
import joblib
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.model import InferenceConfig
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig, Datastore, Environment, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import Pipeline, PipelineData, PipelineRun, TrainingOutput#, StepRun, PortDataReference
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

from azureml.train.hyperdrive import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import quniform, choice
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep

import xgboost as xgboost
xgboost.__version__
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
ws = Workspace.from_config()
experiment_name = 'Custom_Health_Insurance_CS'

exp = Experiment(ws, experiment_name)
def_blob_store = Datastore(ws, "workspaceblobstore")

## Attach computetarget

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "MS-GPUcompute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           max_nodes=8)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

Found existing cluster, use it.
Succeeded........................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Health insurance cross sell"

#dataset = Dataset.get_by_name(workspace, name='Health insurance cross sell')
#dataset.to_pandas_dataframe()

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        dl_data = 'https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction/download'
        dataset = Dataset.Tabular.from_delimited_files(dl_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description="https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction")

#data = dataset.to_pandas_dataframe()

In [5]:
#FOR REMOTE USE
aml_run_config = RunConfiguration()
aml_run_config.target = compute_target
#aml_run_config.environment.docker.enabled = True
#aml_run_config.environment.docker.base_image = "mcr.microsoft.com/azureml/base:latest"

aml_run_config.environment.python.user_managed_dependencies = False

aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn','numpy', 'matplotlib', 'seaborn', 'imbalanced-learn'],
    pip_packages=['joblib','azureml-sdk','phik'],
    pin_sdk_version=False
)



In [6]:
#cd = CondaDependencies(conda_dependencies_file_path='enviroment.yml')
#run_config = RunConfiguration(conda_dependencies=cd)
#run_config.framework = 'python'
#run_config.target = compute_target
#run_config.environment.docker.enabled = True
#run_config.environment.docker.gpu_support = True
#run_config.environment.docker.base_image = "rapidsai/rapidsai-nightly"
#run_config.environment.spark.precache_packages = False
#run_config.data_references={'data':data_ref.to_config()}

In [7]:
myenv = Environment("xgboost") #.from_conda_specification("xgboost", "environment.yml")
myenv.docker.enabled = True
myenv.docker.base_image = "rapidsai/rapidsai-nightly"
conda_dep = CondaDependencies()
conda_dep.add_conda_package("python=3.6")
conda_dep.add_conda_package("pip==19.2.3")
conda_dep.add_conda_package("nb_conda_kernels==2.2.2")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("numpy")
#conda_dep.add_conda_package("xgboost")
conda_dep.add_conda_package("joblib")
conda_dep.add_pip_package("azureml-defaults") #, "prompt_toolkit==2.0.9", "azure-cli==2.0.77", "azureml-core>= 1.18.0"
conda_dep.add_pip_package("xgboost==0.90")
myenv.python.conda_dependencies=conda_dep
myenv.inferencing_stack_version='latest'
#src = ScriptRunConfig(
#                    source_directory='.',
#                    script='train.py',
#                    compute_target=compute_target,
#                    environment=env,
#)

In [8]:
#if "models" not in os.listdir():
#    os.mkdir("./models")
    
raw_data = dataset.as_named_input('raw_data')
X_train = PipelineData("X_train", datastore=def_blob_store).as_dataset()
X_test = PipelineData("X_test", datastore=def_blob_store).as_dataset()
y_train = PipelineData("y_train", datastore=def_blob_store).as_dataset()
y_test = PipelineData("y_test", datastore=def_blob_store).as_dataset()
scaler_file = PipelineData("scaler_file", datastore=def_blob_store)



metrics_output_name = 'metrics_output'
metrics_data = PipelineData(name='metrics_data',
                            datastore=def_blob_store,
                            pipeline_output_name=metrics_output_name,
                            training_output=TrainingOutput("Metrics"))

#model_output_name = 'model_output'
saved_model = PipelineData(name='saved_model', datastore=def_blob_store)
 #,
 #                           pipeline_output_name=model_output_name,
 #                           training_output=TrainingOutput("Model",
 #                                                          model_file="models/best_model.joblib")

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
#Create the different params that you will be using during training
ps = BayesianParameterSampling({
    '--learning_rate': choice(0.1, 0.05),
    '--n_estimators': quniform(1200,2500,200),
    '--max_depth': quniform(12,24,2),
    '--subsample': choice(0.8,0.9),
    '--colsample_bytree': choice(0.8,0.9)
})


# Create a Xgboost estimator for use with train.py
est = Estimator(source_directory='./train/', entry_script='train.py', compute_target=compute_target, environment_definition=myenv)

hyperdrive_run_config = HyperDriveConfig(
                                         hyperparameter_sampling=ps, 
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=2,
                                         max_concurrent_runs=4,
                                        policy=None,
                                        #run_config=run_config
                                        estimator=est
                                        )

'Estimator' is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or an Azure ML curated environment.
For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:100.


In [10]:
source_directory="./prep/"
prep_step = PythonScriptStep(name="prep_step",
                        script_name="prep.py",
                        arguments=["--X_train", X_train, "--X_test", X_test, "--y_train", y_train, "--y_test", y_test, "--scaler", scaler_file],
                        inputs=[raw_data],
                        outputs=[X_train, X_test, y_train, y_test, scaler_file],
                        compute_target=compute_target,
                        runconfig=aml_run_config,
                        source_directory=source_directory,
                        allow_reuse=True)

In [11]:
hd_step = HyperDriveStep(
                        name="hd_step",
                        hyperdrive_config=hyperdrive_run_config,
                        estimator_entry_script_arguments=["--X_train", X_train, "--X_test", X_test, "--y_train", y_train, "--y_test", y_test, "--saved_model", saved_model],
                        inputs=[X_train, X_test, y_train, y_test],
                        outputs=[saved_model])

In [12]:
conda_dep = CondaDependencies()
conda_dep.add_pip_package("azureml-sdk")

rcfg = RunConfiguration(conda_dependencies=conda_dep)

register_model_step = PythonScriptStep(script_name='./register/register_model.py',
                                       name="register_model",
                                       inputs=[saved_model],
                                       compute_target=compute_target,
                                       arguments=["--saved_model", saved_model],
                                       allow_reuse=True,
                                       runconfig=rcfg)

register_model_step.run_after(hd_step)

In [13]:
azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 1000

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
pipeline1 = Pipeline(default_source_directory=".", default_datastore=def_blob_store, workspace=ws, steps=[prep_step, hd_step, register_model_step])
pipeline1.validate()
pipeline_run = Experiment(ws, 'health_insuarance').submit(pipeline1, regenerate_outputs=False)
RunDetails(pipeline_run).show()

SnapshotException: SnapshotException:
	Message: ====================================================================

While attempting to take snapshot of ./train/
Your total snapshot size exceeds the limit of 0.00095367431640625 MB.
Please see http://aka.ms/aml-largefiles on how to work with large files.

====================================================================


	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "====================================================================\n\nWhile attempting to take snapshot of ./train/\nYour total snapshot size exceeds the limit of 0.00095367431640625 MB.\nPlease see http://aka.ms/aml-largefiles on how to work with large files.\n\n====================================================================\n\n"
    }
}

In [ ]:
pipeline_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
model_path = Model.get_model_path("Health_XGBoost_HD", version=None, _workspace=ws)
print(model_path)

In [ ]:
#import joblib
import xgboost
#xgb = xgboost.Booster({'nthread': 1})

#xgb_model = xgb.booster(model_file=model_path)
#xgb.load_model(model_path)

#scaler = np.loadtxt(f"{args.y_test}/y_test.csv", dtype=float)
my_model = joblib.load(model_path)
#model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), '/models/hd_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
my_model

In [ ]:
#from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(entry_script="./score/score.py", environment=myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True)

service = Model.deploy(
    workspace = ws,
    name = "health-insurance-service",
    models = [my_model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)



In [ ]:
#from azureml.core.webservice import Webservice
#
#service_name = 'Health_cross_sell_service'
#service = Webservice.deploy(deployment_config = aciconfig,
#                                image_config = image_config,
#                                model_paths = model_path, #['BestModel.h5'],
#                                name = service_name,
#                                workspace = ws)
#
service.wait_for_deployment(show_output = True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = service.scoring_uri
print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
primary_key, secondary_key = service.get_keys()

service_data = np.loadtxt(f"./output/service_test.csv", dtype=float)
input_data = json.dumps(service_data)
# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {primary_key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.logs

In [ ]:
#service.delete
compute_target.delete()